In [ ]:
def gscott(n, F, k, T, Du=0.16, Dv=0.08, dt=1.0, dx=1.0):
    np.random.seed(seed=7) ## re-initialize seed to keep same initial conditions at each parameter setting
    U, V = np.zeros((n, n), dtype=np.float64), np.zeros((n, n), dtype=np.float64)
    r, m = n//40, n//2
    U[...] = 1.0
    V[m-r:m+r, m-r:m+r] = 0.25
    U[m-r:m+r, m-r:m+r] = 0.5
    U += np.random.normal(scale=0.05, size=U.shape)
    V += np.random.normal(scale=0.05, size=V.shape)
    return gray_scott(U, V, Du, Dv, F, k, dt, dx, T)

def fisher_information(inps,dk,df):
    p = []
    which = [(2,0),(1,1),(1,1),(0,2)]
    for i in inps:
        p.append(gaussian_kde(i))
    matrix = np.zeros(4)
    for i,k in zip(which,range(len(which))):
        matrix[k] = quad(lambda x: (((p[0].logpdf(x)-p[1].logpdf(x))/(2*dk))**i[0])*(((p[2].logpdf(x)-p[3].logpdf(x))/(2*df))**i[1])*p[4].pdf(x), -1000, 10000)[0]
    matrix = matrix.reshape((2,2)) ## Fisher matrix

    return matrix[0,0] + matrix[1,1] ## trace of fisher matrix


def bulk_fisher(f,k,dk,df, n = 100, T = 10000):
    iters = len(f) * len(k)
    
    f, k = np.meshgrid(f, k)
    outp = np.zeros_like(f)
    for index, _f in tqdm(np.ndenumerate(f), total=iters):
        u, v = gscott(n, _f, k[index], T)
        u1,v1 = gscott(n, _f, k[index]+dk, T) ### All different points around point in parameter space
        u2,v2 = gscott(n, _f, k[index]-dk, T)
        u3,v3 = gscott(n, _f+df, k[index], T)
        u4,v4 = gscott(n, _f-df, k[index], T)
        
        obj_u = calc_objects(u)
        obj_u1 = calc_objects(u1)
        obj_u2 = calc_objects(u2)
        obj_u3 = calc_objects(u3)
        obj_u4 = calc_objects(u4)
        
        if obj_u.any() and obj_u1.any() and obj_u2.any() and obj_u3.any() and obj_u4.any():
            inputs = [obj_u1,obj_u2,obj_u3,obj_u4,obj_u]
            outp[index] = fisher_information(inputs,dk,df)
    return outp



f_min = 0.03
f_max = 0.042
k_min = 0.0621
k_max = 0.0631

dims = 4
f = np.linspace(f_min,f_max,dims)
k = np.linspace(k_min,k_max,dims)
df = 0.000000001
dk = 0.000000001

outp = bulk_fisher(f, k,dk,df)

plt.figure(figsize=(15,15))
plt.imshow(np.log10(outp.T), cmap='jet', origin='lower', extent=(k_min,k_max,f_min,f_max), aspect = np.abs((k_max-k_min)/(f_max-f_min)))
plt.xlabel("k")
plt.ylabel("F")
plt.title("Fisher information (10log)")
plt.colorbar()
print(outp.T)
plt.show()